In [1]:
import pandas as pd
import numpy as np
import cdsapi
import utm
import os 
import json
import sys

In [2]:
df = pd.read_excel('ตารางการสำสวจโรคยางพารา.xlsx',sheet_name='พังงา67')

In [3]:
df.head()

,ลำดับ,ชื่อเกษตรกร,วันที่สำรวจ,ตำบล,อำเภอ,จังหวัด,รหัสแปลง(code),พิกัด,Unnamed: 8,อายุต้นยาง(ปี),พันธุ์ยางพารา,โรคที่พบ,เปอร์เซ็นต์การเกิดโรค,ระดับความรุนแรง
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,y,NaN,NaN,NaN,NaN,NaN
1,29.0,นายบุญเกื้อ วัฒนุกิจ,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-1,423961,921346,10,RRIT251,โรคใบร่วงชนิดใหม่,1-10%,1
2,30.0,นายประจิม โสภณ,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-2,424051,921255,6,RRIM600,ไม่พบโรคใบร่วงชนิดใหม่,-,-
3,31.0,นายอนัน คำอ่อน,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-3,424249,921322,9,RRIM600,โรคใบร่วงชนิดใหม่,11-25%,2
4,32.0,นายกระบวน เสือณรงค์,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-4,424315,921264,20,RRIM600,โรคใบร่วงชนิดใหม่,25-50%,3


In [4]:
df = df.iloc[1:,[2,3,4,5,7,8]]

In [5]:
df = df[:].rename({'วันที่สำรวจ':'date','พิกัด':'x','Unnamed: 8':'y'},axis=1)

In [6]:
df

,date,ตำบล,อำเภอ,จังหวัด,x,y
1,02/02/67,นาเตย,ท้ายเหมือง,พังงา,423961,921346
2,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424051,921255
3,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424249,921322
4,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424315,921264
5,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424326,921351
6,02/02/67,ท้ายเหมือง,ท้ายเหมือง,พังงา,419237,933195
7,02/02/67,ท้ายเหมือง,ท้ายเหมือง,พังงา,419193,932922
8,02/02/67,ท้ายเหมือง,ท้ายเหมือง,พังงา,418244,932646


In [7]:
def adjust_year_to_gregorian(date_str):
    date_obj = pd.to_datetime(date_str, format='%d/%m/%y')
    gregorian_year = date_obj.year - 43
    # Recreate the date with the adjusted year
    return date_obj.replace(year=gregorian_year).strftime('%Y-%m-%d')

df['date'] = df['date'].apply(adjust_year_to_gregorian)


In [8]:
df

,date,ตำบล,อำเภอ,จังหวัด,x,y
1,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,423961,921346
2,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424051,921255
3,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424249,921322
4,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424315,921264
5,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424326,921351
6,2024-02-02,ท้ายเหมือง,ท้ายเหมือง,พังงา,419237,933195
7,2024-02-02,ท้ายเหมือง,ท้ายเหมือง,พังงา,419193,932922
8,2024-02-02,ท้ายเหมือง,ท้ายเหมือง,พังงา,418244,932646


In [9]:
def utmToLatLon(x:list, y:list, zone:int = 47) -> tuple:
    x, y = np.mean(x), np.mean(y) #find mid point
    return utm.to_latlon(x,y, zone_number=zone, zone_letter='N') #return (lat, lon)

In [10]:
DIR_OUTPUT = f'../Satun-Phang-Nga/1nc-Phang-Nga/'
os.makedirs(DIR_OUTPUT, exist_ok=True)
c = cdsapi.Client()
for i , data in df.iterrows():
    lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
    year = int(data['date'].split('-')[0])
    month = int(data['date'].split('-')[1])
    day = int(data['date'].split('-')[2])
    print(f"\nDownloading {data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")
    c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                    '2m_temperature', 'downward_uv_radiation_at_the_surface', 'evaporation',
                    'soil_temperature_level_1', 'surface_pressure', 'total_precipitation',
                ],
                'year': year,
                'month': [str(i) for i in range(month-1, month+1)],
                'day': [str(i) for i in range(1, 32)],
                'area' : [lat, lon, lat, lon],                  
                'time': [
                    '00:00', '06:00', '12:00',
                    '18:00',
                ],
            },
            f"{DIR_OUTPUT}{i}-{data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']}.nc")

2024-07-19 16:19:49,502 INFO Welcome to the CDS
2024-07-19 16:19:49,503 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:19:49,835 INFO Request is queued
2024-07-19 16:19:55,488 INFO Request is running
2024-07-19 16:24:24,129 INFO Request is completed
2024-07-19 16:24:24,129 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data3/adaptor.mars.internal-1721380946.7174954-23173-13-90017968-48a7-4aae-8e5b-4ee5146279d2.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/1-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:24:25,848 INFO Download rate 4.8K/s


2024-07-19 16:24:26,441 INFO Welcome to the CDS
2024-07-19 16:24:26,441 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:24:26,806 INFO Request is queued
2024-07-19 16:24:32,450 INFO Request is running
2024-07-19 16:28:49,252 INFO Request is completed
2024-07-19 16:28:49,254 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data9/adaptor.mars.internal-1721381246.4024537-1864-13-2a182b80-a53a-42ab-aa67-ba6e461139a4.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/2-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:28:51,500 INFO Download rate 3.7K/s


2024-07-19 16:28:52,196 INFO Welcome to the CDS
2024-07-19 16:28:52,197 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:28:52,531 INFO Request is queued
2024-07-19 16:28:53,827 INFO Request is running
2024-07-19 16:31:46,784 INFO Request is completed
2024-07-19 16:31:46,785 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data3/adaptor.mars.internal-1721381463.8817785-15995-5-7a174321-d798-4385-8218-ede7028ca4b7.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/3-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:31:49,446 INFO Download rate 3.1K/s


2024-07-19 16:31:50,032 INFO Welcome to the CDS
2024-07-19 16:31:50,033 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:31:50,371 INFO Request is queued
2024-07-19 16:31:53,461 INFO Request is running
2024-07-19 16:33:47,178 INFO Request is completed
2024-07-19 16:33:47,179 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data3/adaptor.mars.internal-1721381614.557815-23503-17-4c8fb481-509a-4919-bbeb-857cd975528c.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/4-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:33:49,763 INFO Download rate 3.2K/s


2024-07-19 16:33:50,353 INFO Welcome to the CDS
2024-07-19 16:33:50,353 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:33:50,698 INFO Request is queued
2024-07-19 16:34:24,958 INFO Request is running
2024-07-19 16:36:44,975 INFO Request is completed
2024-07-19 16:36:44,977 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.mars.internal-1721381768.9217775-22565-4-0e99e808-e1ca-49bd-9094-448ed5838ba9.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/5-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:36:47,020 INFO Download rate 4.1K/s


2024-07-19 16:36:47,617 INFO Welcome to the CDS
2024-07-19 16:36:47,619 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:36:47,921 INFO Request is queued
2024-07-19 16:36:57,245 INFO Request is running
2024-07-19 16:38:44,218 INFO Request is completed
2024-07-19 16:38:44,219 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data8/adaptor.mars.internal-1721381897.1307216-759-8-099336ce-7586-4f24-b37c-1ac636932584.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/6-2024-02-02-ท้ายเหมือง-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:38:46,689 INFO Download rate 3.4K/s


2024-07-19 16:38:47,281 INFO Welcome to the CDS
2024-07-19 16:38:47,282 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:38:47,633 INFO Request is queued
2024-07-19 16:38:48,928 INFO Request is running
2024-07-19 16:40:43,917 INFO Request is completed
2024-07-19 16:40:43,918 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data7/adaptor.mars.internal-1721382026.6460395-2966-6-96872fa0-0830-4c64-b3f2-0f22f7104d29.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/7-2024-02-02-ท้ายเหมือง-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:40:46,579 INFO Download rate 3.1K/s


2024-07-19 16:40:47,177 INFO Welcome to the CDS
2024-07-19 16:40:47,178 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:40:47,542 INFO Request is queued
2024-07-19 16:40:56,848 INFO Request is running
2024-07-19 16:43:41,790 INFO Request is completed
2024-07-19 16:43:41,792 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data4/adaptor.mars.internal-1721382181.7513072-20826-11-03cc7f1e-7f45-41d0-bf82-9a5bb3497f11.nc to ../Satun-Phang-Nga/1nc-Phang-Nga/8-2024-02-02-ท้ายเหมือง-ท้ายเหมือง-พังงา.nc (8.3K)
2024-07-19 16:43:44,196 INFO Download rate 3.5K/s
